In [2]:
from huggingface_hub import notebook_login
notebook_login()

In [6]:
import os
from pathlib import Path
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from typing import List
from langchain_core.documents import Document
import glob

In [8]:

def data_loader(folder_path: str = "data") -> List[Document]:
    pdf_files = glob.glob(os.path.join(folder_path, "*.pdf"))
    all_documents = []

    for pdf_file in pdf_files:
        loader = PyPDFLoader(pdf_file)
        docs = loader.load()
        all_documents.extend(docs)

    return all_documents

In [9]:
documents = data_loader("data")

In [10]:
for i, doc in enumerate(documents[:5]):
    print(f"Document {i+1}:")
    print("Content:", doc.page_content[:200], "...") 
    print("Metadata:", doc.metadata)
    print("-"*60)

Document 1:
Content: THE PUNJAB POLICE RULES
VOLUME 1
Chapter 1
 Organization
Part I 
Departmental Organization
Rule 1.1
1.1.  Constitution. - For the purposes of Section 3 of the Police Act (V of 1861) the Pun-
jab is di ...
Metadata: {'producer': 'Acrobat Distiller 5.0.5 (Windows)', 'creator': 'PScript5.dll Version 5.2', 'creationdate': '2010-05-28T03:57:06+00:00', 'moddate': '2015-08-12T12:32:08+05:30', 'title': 'Punjab Police Rules Volume 1', 'source': 'data\\Punjab-Police-Rules-I.pdf', 'total_pages': 362, 'page': 0, 'page_label': '1'}
------------------------------------------------------------
Document 2:
Content: Finger Print Bureau); a Criminal Investigation Department, and District Police Estab-
lishments.  The Railway Police, general district, is divided into a Central Investigating
Agency, and such number  ...
Metadata: {'producer': 'Acrobat Distiller 5.0.5 (Windows)', 'creator': 'PScript5.dll Version 5.2', 'creationdate': '2010-05-28T03:57:06+00:00', 'moddate': '2015-08-12T

In [11]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
)
document_chunks = text_splitter.split_documents(documents)


In [12]:
embedding_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)
vectorstore = Chroma.from_documents(document_chunks, embedding_model)

C:\Users\ayesh\AppData\Local\Temp\ipykernel_19996\3023741705.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

C:\Users\ayesh\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\ayesh\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [19]:
from langchain.prompts import ChatPromptTemplate
GROQ_API_KEY = os.getenv('GROQ_API_KEY') or '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'
from langchain_groq import ChatGroq
groq_client = ChatGroq(api_key=GROQ_API_KEY, model="llama-3.1-8b-instant")


retriever = vectorstore.as_retriever(search_kwargs={"k": 5})  

In [20]:
chat_prompt = ChatPromptTemplate.from_template(
    """
    You are an expert assistant. Use the following context to answer the user's question. If the answer is not in the context, say you don't know.
    
    Context:
    {context}
    
    Question:
    {question}
    """
)

In [21]:
qa_chain = RetrievalQA.from_chain_type(
    llm=groq_client,
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt": chat_prompt}
)


In [24]:
query = "What is the main topic of Punjab Police Rules?"
result = qa_chain({"query": query})

print("Question:", query)
print("\nAnswer:", result["result"])

Question: What is the main topic of Punjab Police Rules?

Answer: The main topic of Punjab Police Rules appears to be the guidelines and principles governing the organization, functioning, and responsibilities of the police in the Punjab region.
